In [12]:
"""
bsm_model.py
------------
Core Black-Scholes-Merton (BSM) pricing functions for European call/put options.

Formulas (risk-neutral measure):
    d1 = [ln(S/K) + (r - q + 0.5*sigma^2) * T] / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    Call = S * e^{-qT} * N(d1) - K * e^{-rT} * N(d2)
    Put  = K * e^{-rT} * N(-d2) - S * e^{-qT} * N(-d1)

Includes a simple bisection implied volatility solver.
"""
from __future__ import annotations
from dataclasses import dataclass
from typing import Tuple
from math import log, sqrt, exp
import numpy as np
from scipy.stats import norm


@dataclass
class OptionParams:
    S: float       # Spot price
    K: float       # Strike price
    T: float       # Time to maturity (in years)
    r: float       # Risk-free continuously compounded rate (annual)
    sigma: float   # Volatility (annual)
    q: float = 0.0 # Continuous dividend yield (annual)


def _validate(p: OptionParams):
    if p.S <= 0 or p.K <= 0:
        raise ValueError("S and K must be positive.")
    if p.T <= 0:
        raise ValueError("T must be positive (in years).")
    if p.sigma <= 0:
        raise ValueError("sigma must be positive.")


def d1_d2(p: OptionParams) -> Tuple[float, float]:
    """Compute d1 and d2 for BSM with continuous dividend yield q."""
    _validate(p)
    vol_sqrt_t = p.sigma * sqrt(p.T)
    d1 = (log(p.S / p.K) + (p.r - p.q + 0.5 * p.sigma**2) * p.T) / vol_sqrt_t
    d2 = d1 - vol_sqrt_t
    return d1, d2


def price_call_put(p: OptionParams) -> Tuple[float, float]:
    """Return (call_price, put_price) under BSM with dividend yield q."""
    d1, d2 = d1_d2(p)
    disc_r = exp(-p.r * p.T)
    disc_q = exp(-p.q * p.T)
    call = p.S * disc_q * norm.cdf(d1) - p.K * disc_r * norm.cdf(d2)
    put  = p.K * disc_r * norm.cdf(-d2) - p.S * disc_q * norm.cdf(-d1)
    return float(call), float(put)


def implied_vol_bisect(target_price: float, p: OptionParams, option: str = "call",
                       lo: float = 1e-6, hi: float = 5.0, tol: float = 1e-6, max_iter: int = 200) -> float:
    """
    Simple bisection-based implied volatility solver.
    - target_price: observed market price (call or put)
    - p: OptionParams (sigma is ignored and replaced by candidate values)
    - option: "call" or "put"
    Returns implied sigma (annual).
    """
    option = option.lower()
    if option not in ("call", "put"):
        raise ValueError("option must be 'call' or 'put'")

    def price_with_sigma(sig: float) -> float:
        pp = OptionParams(**{**p.__dict__, "sigma": sig})
        c, put = price_call_put(pp)
        return c if option == "call" else put

    v_lo = price_with_sigma(lo) - target_price
    v_hi = price_with_sigma(hi) - target_price
    if v_lo * v_hi > 0:
        raise ValueError("Bisection bracket does not straddle the root. Adjust lo/hi.")

    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        vm = price_with_sigma(mid) - target_price
        if abs(vm) < tol:
            return mid
        if v_lo * vm < 0:
            hi = mid
            v_hi = vm
        else:
            lo = mid
            v_lo = vm
    return 0.5 * (lo + hi)